## Langchain Basic Usage (LLM + ChatModel only)
Langchain的框架下面有主要分為兩種Model
- LLM
- Chat

由於Langchain並不提供任何Model，所以LLM相關的library都不在langchain-core裡面

#### Langchain最基本的用法
`Prompt -> LLM -> Response`

##### Loading environment variables

In [1]:
%load_ext dotenv
%dotenv

import os

##### Basic Prompts

In [2]:
# Everything starts with LLM
from langchain_core.messages import HumanMessage
msg = HumanMessage(content='茶餐廳是什麼？')

##### 使用ChatModel (e.g. TogetherAI)

In [3]:
from langchain_together import ChatTogether
model = ChatTogether(
    together_api_key=os.environ['KEY_TOGETHERAI'],
    model="Qwen/Qwen1.5-72B-Chat",
)

In [4]:
res = model.invoke([msg])

##### 利用output_parser將output變得更易讀

In [5]:
print(res)

content='茶餐廳是源於香港的一種特色餐飲場所，結合了中西餐飲文化，提供多種類、價格實惠的食品和飲品。茶餐廳的菜單通常包括港式茶點、快餐、粥品、面食、西式餐點、甜品等，深受當地居民和遊客的喜愛。\n\n茶餐廳的特色還包括其獨特的裝修風格，通常具有濃厚的本土色彩和一定的時代感。此外，茶餐廳的服務方式也比較隨意和快速，顧客可以點選單上的菜品，並且通常提供茶水自取的服務。\n\n茶餐廳在港式文化中扮演著重要角色，不僅是人們進食的地方，也是社交、商務談話和新聞交流的場所，反映了香港的生活節奏和文化特色。' response_metadata={'token_usage': {'completion_tokens': 177, 'prompt_tokens': 23, 'total_tokens': 200}, 'model_name': 'Qwen/Qwen1.5-72B-Chat', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None} id='run-7fb6f0f5-f794-4302-984f-0f1be04edb86-0' usage_metadata={'input_tokens': 23, 'output_tokens': 177, 'total_tokens': 200}


In [6]:
from langchain_core.output_parsers import StrOutputParser

p = StrOutputParser()
# print(type(p))
print(p.invoke(res))

茶餐廳是源於香港的一種特色餐飲場所，結合了中西餐飲文化，提供多種類、價格實惠的食品和飲品。茶餐廳的菜單通常包括港式茶點、快餐、粥品、面食、西式餐點、甜品等，深受當地居民和遊客的喜愛。

茶餐廳的特色還包括其獨特的裝修風格，通常具有濃厚的本土色彩和一定的時代感。此外，茶餐廳的服務方式也比較隨意和快速，顧客可以點選單上的菜品，並且通常提供茶水自取的服務。

茶餐廳在港式文化中扮演著重要角色，不僅是人們進食的地方，也是社交、商務談話和新聞交流的場所，反映了香港的生活節奏和文化特色。


##### 加入SystemMessage (Instruction)

In [7]:
from langchain_core.messages import SystemMessage
system_message = """
必須以繁體中文回答
回答時只以','分隔每個項目，並以最簡短的方式回答，例如：
一,二,三,...
"""
human_message = """
請列舉茶餐廳不多過10種最熱門的食物，不肯定的不要回答
"""
msg_list = [
    SystemMessage(content=system_message),
    HumanMessage(content=human_message)
]

In [8]:
res = model.invoke(msg_list)

In [9]:
print(p.invoke(res))

蛋撻,菠蘿油,奶茶,火腿煎蛋三明治,豬扒包,炸雞翼,雲吞面,羅宋湯,魚蛋,咖喱牛腩


##### Structured Data Parser
https://python.langchain.com/v0.2/docs/concepts/#output-parsers

In [10]:
from langchain.output_parsers import CommaSeparatedListOutputParser

csvp = CommaSeparatedListOutputParser()
parsed_res = csvp.invoke(res)
print(type(parsed_res))
print(parsed_res)

<class 'list'>
['蛋撻', '菠蘿油', '奶茶', '火腿煎蛋三明治', '豬扒包', '炸雞翼', '雲吞面', '羅宋湯', '魚蛋', '咖喱牛腩']


##### 用PromptTemplate更方便

In [11]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", "請列舉{place}不多過10種最熱門的食物，不肯定的不要回答"),
])

prompt_value = prompt_template.invoke(
    {
        'place': '日本'
    }
)

In [12]:
print(prompt_value)

messages=[SystemMessage(content="\n必須以繁體中文回答\n回答時只以','分隔每個項目，並以最簡短的方式回答，例如：\n一,二,三,...\n"), HumanMessage(content='請列舉日本不多過10種最熱門的食物，不肯定的不要回答')]


In [13]:
japan_csv = csvp.invoke(model.invoke(prompt_value))

In [14]:
print(japan_csv)

['寿司', '拉麵', '天婦羅', '燒肉', '丼飯', '章魚燒', '炸豬排', '刺身', '懷石料理', '便當']


##### 以steaming的方式回答

In [15]:
author_system_message = """你是一個日本輕小說作家，寫作風格要模仿{author}，請以用戶的名字命名主角，並以提供的關鍵字去寫一個大約400字的短篇故事。故事必須以繁體中文編寫。"""
prompt_template2 = ChatPromptTemplate.from_messages([
    ("system", author_system_message),
    ("ai", "請問你怎樣稱呼？"),
    ("human", "我的名字叫{username}"),
    ("ai", "你好{username}，請問你想以什麼作題材寫這個故事呢？"),
    ("human", "我想以{topics}為題材寫這個故事"),
])
prompt_value = prompt_template2.invoke(
    {
        "author": "川原礫",
        "username": "彌豆子",
        "topics": "鬼，捉鬼，親情，拯救世界"
    }
)
print(prompt_value)

messages=[SystemMessage(content='你是一個日本輕小說作家，寫作風格要模仿川原礫，請以用戶的名字命名主角，並以提供的關鍵字去寫一個大約400字的短篇故事。故事必須以繁體中文編寫。'), AIMessage(content='請問你怎樣稱呼？'), HumanMessage(content='我的名字叫彌豆子'), AIMessage(content='你好彌豆子，請問你想以什麼作題材寫這個故事呢？'), HumanMessage(content='我想以鬼，捉鬼，親情，拯救世界為題材寫這個故事')]


In [16]:
for res in model.stream(prompt_value):
    print(res.content, end="|")

故事|標|題|：|《||彌|豆|子|的|鬼|之|挽|歌|》|

|在|一個|被|鬼|魂|侵||蝕|的世界|中|，|人類|生活在|恐||懼|與|陰|影|之下|。||彌|豆|子|，|一名|看似|普通的|少女|，|卻|是|古老的|捉|鬼|世家|之後|裔|。|她的|家族|蒙|受||詛|咒|，|每|一代|都|必須|承|擔|起|對|抗|鬼|怪|的|使命|。|而||彌|豆|子|，|自|小|便|展|現|出|異|於|常|人的|力量|，|她的|血液|對|鬼|物|有|致命|的|威||懾|。

|一天|，|最|強|惡|鬼|"|冥|王|"|破|封|而出|，|吸取|了|世界的|光|亮|，|將|黑夜|永||恆|化|。|為了|拯救|即|將|崩||潰|的世界|，||彌|豆|子|毅然|決|然|踏上|旅程|。|她|帶著|父親|留|下的|古|舊|符|咒|和|一把|刻|著|家族|纹|章|的|短|刀|，|踏|上了|捉|鬼|之路|。

|在|旅|途中|，||彌|豆|子|遇到了|各式|各|樣|的|同伴|，|他們|的|目標|與|她|相同|，|都是|為了|結束|這個|黑暗|的|時代|。|他們|共同|經歷|生死|，|彼此|的|友||誼|和|信|賴|逐||漸|加深|。|然而|，|隨著|力量|的|覺|醒|，||彌|豆|子|也|逐||漸|發現|家族|的秘密|——|她的|存在|可能|才是|冥|王|降|臨|的|關鍵|。

|在|最終|的|決|戰|中|，||彌|豆|子|為了|保護|世界|，|選擇|與|冥|王|正面|對|決|。|亲情|的力量|激|發|了|她的|潛|能|，|她|將|血液|的力量|注入|短|刀|，|發|動|了|家|傳|符|咒|的|終|極|技|"|鬼|之|挽|歌|"|。|一|瞬|間|，|光芒|刺|破|黑暗|，|冥|王|在|光芒|之下|灰|飛|烟|滅|。

|當|曙光|再次|普|照|大地|，|人們|從|陰|霾|中|蘇|醒|，|世界|恢|復|了|和平|。||彌|豆|子|的名字|成為|了|传说|，|而|她的|勇|氣|與|愛|，|永遠|照亮|了|這個|世界|。|然而|，|她|選擇|淡|出|人們|的|視|線|，|回到|平凡|的生活|，|繼續|守|護|著|她的|家人|和|朋友|，|默默|等待|下|一個|黑暗|的|來|臨|。||

#### LCEL: LangChain Expression Language
怎樣用LCEL簡化日本輕小說家的Chain呢？ 

In [17]:
jnovel = prompt_template2 | model | p
res = jnovel.invoke(
    {
        "author": "支倉凍砂",
        "username": "亨利",
        "topics": "狼，香辛料，北方，旅行商人"
    })

In [18]:
print(p.parse(res))

故事標題：《亨利的北方香料之旅》

在遙遠的北方之地，有一位名叫亨利的旅行商人。他獨自駕駛著覆著厚厚毛皮的雪橇，穿越冰冷的風雪，帶著貨物來往於各個村落。亨利不僅販賣實用的工具和布料，更以一種罕見的香辛料聞名遐邇，這種香料能驅走嚴寒，為人們帶來家的溫暖。

一次，亨利來到了一個被狼群包圍的偏遠小村。村裡的人們生活在恐懼中，因為狼群的攻擊日益頻繁。亨利決定用他的香辛料作為交換，教導村人一種能安撫野獸的特別調料，希望能夠化解這次危機。他 állapota, hogy a farkasokkal való beszélgetés során, felfedezte, hogy a vezető farkas, egy idős és bölcs állat, a falusiak lelki fájdalmát érzékeli.

亨利, a különleges fűszerek mestere, összekeveri a rejtélyes ízeket, és egy csodás ételest elkészít. A falusiak megkóstolják, majd a távolemente előtt, a vezető farkast is meghívják. Az állat megkóstolja az étel egy részét, és a közös étkezés közben valami változik a két világ között - a farkasok lassan távolodnak.

亨利 ezzel bizonyította, hogy a barátság és megértés sokkal erősebb, mint az erőszak. Azóta, a farkasok nem zaklatják többé a falut, és亨利 neve a térségben még nagyobb tisztelet övezi. Ő továbbra is utazik, hogy más helyeken is segítse az embereket, de mindig emlékszik vissza arra, hogyan egy csokoládéből és herbolári

In [19]:
jnovel2 = prompt_template2 | model | p
for res in jnovel2.stream(
    {
        "author": "支倉凍砂",
        "username": "亨利",
        "topics": "狼，香辛料，北方，旅行商人"
    }):
    print(res, end="")

故事標題：《亨利的北方香辛料之旅》

在遙遠的北方，有一位名叫亨利的年輕旅行商人，他以贩卖稀有的香辛料為生。亨利的旅程總是充滿未知，而他的獨特眼光和敏銳味蕾，讓他能發掘出最純正的香料。一天，亨利聽說在北地的一個被雪覆蓋的村莊，發現了一種傳說中的香辛料——「銀狼之息」，它據說能調出能暖人心扉的味道。

亨利決定踏上這趟冒險之旅，穿越凜冽的風雪，來到冰雪覆蓋的村莊。在那裡，他遇到了一隻獨特的狼，其眼神深邃，毛皮閃耀著銀色的光芒。亨利驚訝地發現，這隻狼對香辛料有著敏銳的嗅覺，它成為亨利尋找「銀狼之息」的關鍵伙伴。

他們一同穿過冰封的森林，面對寒冷與危險，狼的智慧和亨利的決心讓他們屢次化險為夷。終於，在一個被風雪掩藏的洞穴中，他們找到了「銀狼之息」，它的氣息果然如傳說中般，既深邃又溫暖，彷彿冬日裡的一抹暖陽。

亨利帶著這份寶貴的收穫返回南方，他的故事成為了各地商人和旅人的佳話。而那隻銀狼，也選擇跟隨亨利，成為他的守護者，陪伴他在各地的香辛料之旅中，繼續編織著更多的傳奇故事。

##### 問題：Print的為何不再是res.content?

https://python.langchain.com/v0.2/docs/how_to/streaming/

### 偉大的LangChain: Runnable
https://python.langchain.com/v0.2/docs/concepts/#runnable-interface

#### Bonus Track: Tools Calling 😎

In [20]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

tools = [add, multiply]

In [21]:
from langchain_core.pydantic_v1 import BaseModel, Field

# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

class Multiply(BaseModel):
    """Multiply two integers together."""
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [22]:
# mistralai/Mixtral-8x22B-Instruct-v0.1
from langchain_openai import ChatOpenAI

better_model = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["KEY_TOGETHERAI"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",)

model_with_tools = better_model.bind_tools(tools)

In [23]:
query = "What is 3 * 12? Also, what is 11 + 49?"

model_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_9bluxix7ssmxnqfv787ha0rw'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_v04c4cdnk7aqpm3j8yf03j98'}]

In [26]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(query)]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [27]:
print(messages)

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0u8nt9yy4fd1bsankb2daxzh', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_e2p56b3w6wff4rtka6s0sz7r', 'function': {'arguments': '{"a":11,"b":49}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 304, 'total_tokens': 388}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-294eff05-079f-4c98-9cb9-0c532453b531-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_0u8nt9yy4fd1bsankb2daxzh'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_e2p56b3w6wff4rtka6s0sz7r'}], usage_metadata={'input_tokens': 304, 'output_tokens': 84, 'total_tokens': 388}), ToolMessage(content='36', tool_call_id='call_0u8nt9yy4fd1bsankb2daxzh'), ToolM

In [31]:
model_with_tools.invoke(messages)

AIMessage(content='', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 436, 'total_tokens': 437}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-79a62648-2476-4ce5-9fd4-56b25c0d89dd-0', usage_metadata={'input_tokens': 436, 'output_tokens': 1, 'total_tokens': 437})